## **LangChain -  FAISS**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [ ]:
# 1) load PDF
pdf_path = 'Hayat-Qulub-Alama-Majlisi.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [ ]:
print(loader)

In [ ]:
# 2) Split into chunks 
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)

In [ ]:
# 3-A) Create Embeddings - Sentenace Transformer Embeddings
embeddings = HuggingFaceBgeEmbeddings(model_name='all-MiniLM-L12-v2', model_kwargs={'device':'cuda'})
embeddings

In [ ]:
# 4) Store Vector Embeddings
vectordb = FAISS.from_documents(chunks, embeddings)
vectordb.save_local("faiss_all_minilm")

In [ ]:
# 5) Retrieve 
retrieve = vectordb.as_retriever(search_type = 'similarity', search_kwargs={"k":10})
retrieve

In [ ]:
from dotenv import load_dotenv
# ---------- Load env var ----------
load_dotenv()

API_KEYS = os.getenv("GEMINI_API_KEYS", "").split(",")
API_KEYS = [k.strip() for k in API_KEYS if k.strip()]
current_key_index = 0
ACCESS_KEY = os.getenv("APP_ACCESS_KEY")

if not API_KEYS:
    print("❌ No Gemini API keys found. Check .env file.")
    raise Exception("Missing GEMINI_API_KEYS")

In [ ]:
# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
If the answer is not in the documents, say "I don’t know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=system_prompt
)

In [ ]:
# 6) LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    api_key=os.getenv('GEMINI_KEY_KEY', '')
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retrieve,
    return_source_documents=False,
    chain_type_kwargs = {"prompt":prompt}
)

In [ ]:
# 7) Ask
query = "Why Adam was named Adam? Give reference to it as well"
res = qa.run(query)
print(res)

In [ ]:
q1 = "Why Adam was named Adam? Explain in details"
res = qa.run(q1)
print(res)

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

## **Pinecone Storage**

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
import os
from dotenv import load_dotenv

In [9]:
load_dotenv()
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(INDEX_NAME)


### **Vectorinze Single PDF**

In [20]:
# ---------- begin of pipelines ----------
# load pdf
pdf_path = 'Hayat-al-Qulub-Vol-1.pdf'
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

ValueError: File path Hayat-al-Qulub-Vol-1.pdf is not a valid file or url

In [ ]:
# split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)

In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)

In [ ]:
# Store in vector database - FAISS
vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("Hayat-Qulub-Alama-Majlisi-faiss-index")

### **Vectorize multiple PDFs**

In [16]:
#  Store all 3 PDFs in vector database - Pinecone
pdfs = [
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-1.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-2.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-3.pdf'
]

all_chunks = []
for i, pdf in enumerate(pdfs):
    loader = PyMuPDFLoader(pdf)
    docs = loader.load()
    print(f"Loaded {i} documents")
    
    # split text into chunks
    print(f"Splitting {i} documents into chunks...")
    splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    chunks = splitter.split_documents(docs)
    print(f"splitted {i} documents Successfully! ")
    print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

    all_chunks.extend(chunks)
    print(f"Chunk list filled")
    print(f"Total chunks so far: {len(all_chunks)}")
    


Loaded 0 documents
Splitting 0 documents into chunks...
splitted 0 documents Successfully! 
Loaded 2547 chunks from 0 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-1.pdf
Chunk list filled
Total chunks so far: 2547
Loaded 1 documents
Splitting 1 documents into chunks...
splitted 1 documents Successfully! 
Loaded 3806 chunks from 1 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-2.pdf
Chunk list filled
Total chunks so far: 6353
Loaded 2 documents
Splitting 2 documents into chunks...
splitted 2 documents Successfully! 
Loaded 1205 chunks from 2 document hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-3.pdf
Chunk list filled
Total chunks so far: 7558


In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)


In [ ]:
# Store in vector database - Pinecone
vector_db = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
pinecone_store = vector_db

In [19]:
pinecone_store = vector_db

### **Vectorize multiple PDFs - functions**

In [ ]:
def process_pdf_into_chunks(pdfs):
    pdfs = list(pdfs)
    all_chunks = []
    for i, pdf in enumerate(pdfs):
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} documents")
        
        # split text into chunks
        print(f"Splitting {i} documents into chunks...")
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"splitted {i} documents Successfully! ")
        print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

        all_chunks.extend(chunks)
        print(f"Chunk list filled")
        print(f"Total chunks so far: {len(all_chunks)}")
    
    return all_chunks

def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
    # create embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model, model_kwargs={"device": "cuda"}
    )
    print(f"Created embeddings using model: {embedding_model}")
    
    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store


### **Retrieve QA**

In [9]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cpu"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
You have to understand the context deeply and answer accordingly.
If the answer is not in the documents, say "I don't know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())


In [ ]:
# Ask a question
query = "Why Adam was named Adam? Give reference to it as well"
res = qa_chain.invoke(query)

print(res)

In the tapestry of Islamic history, the naming of Adam is a tale woven with profound meaning.

It is narrated through authentic chains that Imam Muhammad al-Baqir and Ja‘far as-Sadiq (peace be upon them) said, "Adam was named ‘Adam’ because he was ‘Adeemul Arz’, that is he was created from the face of the earth (from dust)."

So, Adam's name is intrinsically linked to his origin, a reminder of the humble earth from which he was formed.


In [13]:
query1 = """
Kisne likh hai Hayatal Qulub? kiske bare me hai ye kitab? 
kitni jildhe hai iske aur kya har ek jildh ko short me smjha sakte ho?
"""
res2 = qa_chain.invoke(query1)
print(res2)

Aapka sawaal Hayat-ul-Qulub ke baare mein hai. Chaliye, main aapko iske baare mein tafseel se batata hoon:

**Hayat-ul-Qulub kisne likhi hai?**

Hayat-ul-Qulub Allama Muhammad Baqir Majlisi (Rehmatullah Alaih) ne likhi hai. Allama Majlisi ek mashhoor Islamic scholar aur Muhaddith the.

**Ye kitab kiske baare mein hai?**

Ye kitab Ambiya (Prophets), Aimma (Imams), aur deegar buzurg hastiyon ke ahwaal (life events) aur fazail (virtues) par mabni hai. Isme unke zindagi ke waqiyat, unke akhlaq, aur unke maqamat ko tafseel se bayan kiya gaya hai.

**Is kitab ki kitni jildhein (volumes) hain?**

Hayat-ul-Qulub ki teen jildhein hain. Har jildh mein mukhtalif anbiya aur aimma ke baare mein tafseeli malumaat hain.

**Har jildh ka mukhtasar khulasa (brief summary):**

*   **Jildh 1:** Is jildh mein Ambiya-e-Kiram (Prophets) ke ahwaal bayan kiye gaye hain, jaise Hazrat Adam (A.S.), Hazrat Nuh (A.S.), Hazrat Ibrahim (A.S.), Hazrat Musa (A.S.), aur Hazrat Isa (A.S.). Har Nabi ki zindagi ke aham waq

In [ ]:
# quick check for any embedding wrapper you plan to use
vec = embeddings.embed_query("test")
print("type:", type(vec))
print("len:", len(vec))   # MUST be 1024 to match your Pinecone index

## **Sulaym ibne Qays RAG**

### **RAG Pipeline**

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
from dotenv import load_dotenv
import os

c:\Users\sadiq\OneDrive\Documents\projects\rag-for-Hayatal-Qulub\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [6]:
load_dotenv()

pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ")
index = pc.Index(INDEX_NAME)

In [33]:
def process_pdf_into_chunks(pdfs: list):
    print(f'Count of Documents - {len(pdfs)}')

    all_chunks = []
    for i, pdf in enumerate(pdfs):
        i=+1
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} document successfully")

        # split into chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"Splitted {i} documment successfully!")

        all_chunks.extend(chunks)
        print(f"Total chunks so far: {len(all_chunks)}")

    return all_chunks
    
def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
     #create embeddings

    embeddings = HuggingFaceEmbeddings(
        model_name = 'BAAI/bge-large-en-v1.5', model_kwargs={"device": "cuda"} 
        )
    print(f"Created embeddings using model: {embedding_model}")

    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        index_name=INDEX_NAME
    )
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store
    

In [34]:
chunks = process_pdf_into_chunks(pdfs=['kitab_sulaim_ibn_qays_al-hilaali.pdf'])

Count of Documents - 1
Loaded 1 document successfully
Splitted 1 documment successfully!
Total chunks so far: 753


In [35]:
vector_store = store_chunks_in_pinecone(chunks, 'BAAI/bge-large-en-v1.5', INDEX_NAME)

Created embeddings using model: BAAI/bge-large-en-v1.5
Stored 753 chunks in Pinecone index: sulaym-ibne-qays


### **Retrieve QA**

In [1]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from system_prompt import system_prompt_text
load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cpu"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = system_prompt_text

prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())


d:\Sadiq\rag-for-hq-books\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Sadiq\rag-for-hq-books\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [2]:
from IPython.display import display, Markdown
# Ask a question
query = "Few people attacked the house of Lady Fatimah (SA) and burned the door. what was the entire incident? Explain in details. also provide references like hadith no and page."
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that after the Holy Prophet (SAW) passed away, Umar went to Ali's (AS) house to force him to pledge allegiance to Abu Bakr.

Umar came until he knocked on the door and called “O ibn Abi Talib, open the door.” 
Fatimah (AS) said: “O Umar, what do we have to do with you? Why do not you leave us as we are?” 
He replied: “Open the door otherwise we will burn it for you.” 
She said: “O Umar, do not you fear Allah Azz Wa Jall? You are entering my house and are surrounding my building.” He refused to go. Then he sent for fire, and set the door alight and then Umar pushed it. Then Fatimah (AS) came in front of him and screamed; “O father, O Messenger of Allah!” So he (Umar) raised his sword which was in the cover and hurt her in her side. She screamed, and he lifted whip and hit her hand. She screamed; “O father!” 
Ali ibn Abi Talib (AS) ran and held Umar by his neck. Then he moved him and made him fall and twisted his nose and neck and intended to kill him. At that time he remembered what the Holy Prophet (SAW) had told him and mentioned in the will for patience and obedience, and said: “By Him, who held Muhammad high with Prophethood, O ibn Sahhak, if this written thing had not passed, then you would have known that you cannot enter my house.” 
Umar sent someone for help. People came and entered the house. Khalid ibn Walid lifted the sword to hit Fatimah (AS), so Ali (AS) attacked him. He held Ali (AS) under oath, so he stopped. 
Miqdad, Salman, Abu Dhar, AMmar, and Buraydah Aslami came and entered the house to help Ali (AS). It was very near that fight would start. So Ali (AS) reminded them of the will of the Holy Prophet (SAW) and their oath, so they returned. (Pages 863-864)

According to the traditions, Janabe Sulaym ibne Qays narrated from Salman al-Farsi that Umar, after being refused entry by Lady Fatimah (SA), threatened to burn the house down if Imam Ali (AS) did not come out and pledge allegiance to Abu Bakr.

...Umar told those people who had gathered around him to collect wood. They all collected, and Umar himself lifted and went to the house of Hadhrat Ali (AS), Lady Fatimah (AS) and their two sons, and arranged wood all around the house and then said in a voice loud enough to make Hadhrat Ali (AS) and Lady Fatimah (AS) hear: “By God, O Ali, come out and pay allegiance to the caliph of the Holy Prophet of Allah, otherwise we will burn your house.” 
 
Lady Fatimah (AS) said: “O Umar, what do you have to do with us?” 
 
He replied: “Open the door, otherwise we will burn your house.” 
 
Lady Fatimah (AS) said: “O Umar, are you not afraid of Allah and are you entering our house?” (Page 860)

According to the traditions, Janabe Sulaym ibne Qays narrated from Salman al-Farsi that Umar and his companions forcibly entered the house of Lady Fatimah (SA) and mistreated her.

...They returned, but Qunfuz, the cursed, remained. His companions told that Lady Fatimah had said such and such, and she was not permitting them to enter. Umar said angrily: “What do we have to do with women?” 
 
Then Umar told those people who had gathered around him to collect wood. They all collected, and Umar himself lifted and went to the house of Hadhrat Ali (AS), Lady Fatimah (AS) and their two sons, and arranged wood all around the house and then said in a voice loud enough to make Hadhrat Ali (AS) and Lady Fatimah (AS) hear: “By God, O Ali, come out and pay allegiance to the caliph of the Holy Prophet of Allah, otherwise we will burn your house.” 
 
Lady Fatimah (AS) said: “O Umar, what do you have to do with us?” 
 
He replied: “Open the door, otherwise we will burn your house.” 
 
Lady Fatimah (AS) said: “O Umar, are you not afraid of Allah and are you entering our house?” 
 
Umar refused to return. He asked fire to be brought and he set the door on fire, then he pushed it and entered. Lady Fatimah (AS) came in front and screamed loudly: “O Father, O Prophet of Allah.” 
 
Umar raised his sword with the shield and hit her on the side. She screamed: (Pages 860-861)

According to the traditions, Janabe Sulaym ibne Qays narrated from Salman al-Farsi that after the incident, Lady Fatimah (SA) lamented the actions of Abu Bakr and Umar.

Sulaym says: “I asked Salman: “Did these people evter the house of Lady Fatimah (AS) without permission?” He replied: “Yes, by God, when she did not even have a chaddar over her. So she screamed: “O Father, O Prophet of Allah, Abu Bakr and Umar behaved so badly after you, while your eyes have not even closed in the grave” and she was saying this loudly.” (Page 865)

In [3]:
query = "Explain the Hadith no 1 exactly in detail"
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that Abu Amir Al Bara ibn Adhib-al Ansari Al Khadhraji, who participated in 14 wars along with the Holy Prophet (PBUH), said:

1. "I did not think that this matter (caliphate) would pass away from Bani Hashem, and from Abul Hasan.
2. Is he not the first person to recite prayer in the direction of your Qibla? And is he not the one who knows, better than everyone, all Ahadith and Traditions?
3. And he is nearest to the Holy Prophet (SAW) and he is the one who was helped by Jibra’eel in the bathing and shrouding.
4. And he is the one who has everything that people have, but people do not have merits that he has.
5. Ali (AS) is he, and for what you kept away this (caliphate) from Him, I know it. This allegiance of yours is the first fitnah." (Page 14)

This narration from Kitab Sulaym ibn Qays highlights the sentiments of Abu Amir Al Bara ibn Adhib-al Ansari Al Khadhraji, a companion of the Prophet Muhammad (PBUH) who fought in 14 battles alongside him. It expresses his surprise and disapproval at the caliphate not being given to Imam Ali (AS) after the Prophet's (PBUH) passing.

The narration emphasizes several key points:

*   **Lineage and Right to Caliphate:** Abu Amir expresses his astonishment that the caliphate was diverted from Bani Hashem, the Prophet's (PBUH) clan, and specifically from Imam Ali (AS), referred to as "Abul Hasan." This suggests a belief that the Prophet's (PBUH) family had a natural or divinely ordained right to leadership.
*   **Precedence in Islam:** He highlights Imam Ali's (AS) early embrace of Islam, noting that he was the first to pray towards the Qibla (direction of prayer). This underscores Imam Ali's (AS) dedication and commitment to Islam from its inception.
*   **Knowledge and Authority:** Abu Amir emphasizes Imam Ali's (AS) unparalleled knowledge of the Ahadith (sayings and traditions of the Prophet) and traditions. This suggests that Imam Ali (AS) was uniquely qualified to lead the Muslim community due to his deep understanding of Islamic teachings.
*   **Proximity to the Prophet (PBUH):** He points out Imam Ali's (AS) close relationship with the Prophet (PBUH), mentioning that he was the one assisted by Jibra'eel (Gabriel) in the bathing and shrouding of the Prophet's (PBUH) body. This highlights Imam Ali's (AS) special status and the trust placed in him by the Prophet (PBUH).
*   **Unique Merits:** Abu Amir asserts that Imam Ali (AS) possesses all the qualities that others have, but others do not possess the unique merits that he has. This suggests that Imam Ali (AS) was superior to others in terms of virtue, knowledge, and leadership qualities.
*   **Rejection of Imam Ali (AS) as a Fitnah:** Finally, Abu Amir states that he knows the reason why the caliphate was kept away from Imam Ali (AS) and declares the allegiance given to others as the "first fitnah" (trial or tribulation). This indicates his belief that the decision to exclude Imam Ali (AS) from leadership was a grave mistake that led to discord and division within the Muslim community.

In essence, this narration reflects the perspective of a devoted companion of the Prophet (PBUH) who believed that Imam Ali (AS) was the most deserving candidate for leadership after the Prophet's (PBUH) death and that the decision to bypass him was a significant deviation from the right path.

In [5]:
query = "Who are those 3 types of people who understand and accept the Ahle bait (as)?"
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali (AS) said during the month of Ramadhan, and this is that month in which he was martyred, when he was between his two sons Hasan and Husain (AS) and amid the children of Abdullah ibn Ja’afer ibn Abi Talib and his special Shias. He said: "Leave people to what they want for themselves. You people choose silence when it is your enemy’s kingdom, because with what ever you are connected to will not remain secret from them and the enemy is oppressor and jealous.

People are of three types. One type is illuminated with our light (Noor), and one type is eating through us and one is that who has been guided through us and obeyed our Amr. These are very few in this type. These are those who are Shias, noble, people of patience, are learned, jurist, pious and generous. For them there is happiness and also a tree in heaven, and wherever they go, the place will be very good.” (Page 151)

In [43]:
# Ask a question
query = """
Give me 5 merits of Imam Ali (AS) in Islam with references.
"""
res = qa_chain.invoke(query)
print(res)

From the narrations in Kitab Sulaym ibn Qays, the merits of Imam Ali (AS) are numerous and profound. Here are five, illuminated by the traditions within this esteemed collection:

1.  **The First to Accept Islam:** Imam Ali (AS) was the first among the Ummah to embrace Islam. As Imam Ali (AS) himself states, referencing the verse "{'And the foremost are the foremost, These are drawn nigh (to Allah)' } (Surah Waqia: 10- 11)," no one preceded him in turning towards Allah and His Prophet (PBUH). This precedence is not merely a matter of historical record, but a testament to his unwavering faith and devotion from the earliest days of Islam.

2.  **Unmatched Knowledge and Jurisprudence:** After the passing of the Holy Prophet (PBUH), all were dependent on Imam Ali (AS) for knowledge and jurisprudence, while he remained independent of all others. The Holy Prophet (PBUH) himself imparted knowledge to Imam Ali (AS) privately, day and night, ensuring that he was deeply versed in the Book of All

In [44]:
from IPython.display import display, Markdown
display(Markdown(res))

From the narrations in Kitab Sulaym ibn Qays, the merits of Imam Ali (AS) are numerous and profound. Here are five, illuminated by the traditions within this esteemed collection:

1.  **The First to Accept Islam:** Imam Ali (AS) was the first among the Ummah to embrace Islam. As Imam Ali (AS) himself states, referencing the verse "{'And the foremost are the foremost, These are drawn nigh (to Allah)' } (Surah Waqia: 10- 11)," no one preceded him in turning towards Allah and His Prophet (PBUH). This precedence is not merely a matter of historical record, but a testament to his unwavering faith and devotion from the earliest days of Islam.

2.  **Unmatched Knowledge and Jurisprudence:** After the passing of the Holy Prophet (PBUH), all were dependent on Imam Ali (AS) for knowledge and jurisprudence, while he remained independent of all others. The Holy Prophet (PBUH) himself imparted knowledge to Imam Ali (AS) privately, day and night, ensuring that he was deeply versed in the Book of Allah and the traditions of the Prophet (PBUH).

3.  **Bravest in Wars and Defender of the Prophet (PBUH):** Imam Ali (AS) was always at the forefront, trusted by the Holy Prophet (PBUH) due to his virtues. No difficulty befell the Holy Prophet (PBUH), no problems arose for the Holy Prophet (PBUH) defending against bravery, conquering any fort, but Ali was always sent in forefront. He consistently risked his life to protect the Holy Prophet (PBUH) and defend the nascent Muslim community.

4.  **The Brother, Vizier, and Successor of the Prophet (PBUH):** Allah sent a revelation to the Holy Prophet (PBUH) to make Imam Ali (AS) his brother, friend, vizier, successor, and caliph. The Holy Prophet (PBUH) declared that Imam Ali (AS) is, after him, the guardian of all believers. This appointment signifies Imam Ali's (AS) divinely ordained role in guiding the Ummah after the Prophet (PBUH).

5.  **The Organizer, Witness, and Guide of the Ummah:** The Holy Prophet (PBUH) said that Ali (AS) is the organizer of this Ummah and is a witness over it and is responsible for its account, and that is the greatest Sign, and is very illuminated road of truth and is Allah's Path. After the Holy Prophet (PBUH), guidance can be got from him after deviation, and through him can the light be achieved from blindness. This highlights Imam Ali's (AS) role as the guiding light for the Ummah, leading them from darkness to enlightenment.

In [45]:
# Ask a question
query = "Tell me about the 73rd sect which will go to paradise with references."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the narrations in Kitab Sulaym ibn Qays, the matter of the 73 sects is a grave one, filled with both warning and hope. Imam Ali (AS) speaks extensively on this topic, clarifying the path to salvation.

He (AS) states that this Ummah will be divided into 73 sects, with only one destined for Paradise and the remaining 72 for Hell. This echoes the divisions that plagued the communities of Musa (AS) and Isa (AS), where only those who followed the true successors were saved.

The saved sect, as described by Imam Ali (AS), is the one that follows the *wasi* of Prophet Muhammad (PBUH). He (AS) emphasizes that this sect is guided, hopeful, and possesses true *Iman*. They obey commands, are agreeable and intelligent, and trust in him (AS). They recognize his (AS) rights and *Imamat*, understanding that obedience to him (AS) is compulsory according to the Book of Allah and the tradition of the Holy Prophet (PBUH). They do not become apostates and harbor no doubt, for Allah has enlightened their hearts with the knowledge of the Ahlulbayt's rights and merits.

Furthermore, Imam Ali (AS) clarifies that this saved sect will not separate from the Quran, and the Quran will not separate from them until they reach the Fountain of the Holy Prophet (PBUH). They are the Ahlulbayt for whom Ibrahim (AS) prayed, as mentioned in Surah Ibrahim.

However, it is crucial to understand that merely claiming love for the Ahlulbayt is not enough. Imam Ali (AS) states that even among those who profess love and devotion (*Muwaddah*) for him (AS), many will still be misguided. He (AS) says that out of 13 sects who connect themselves to the love of the Ahlulbayt, only one will reach Paradise, while the other 12 will be in Hell.

The key lies in truly understanding and adhering to the *Wilayat* of the Ahlulbayt, recognizing the misguidance of their enemies, and upholding what is lawful and unlawful according to their teachings. This saved sect chooses Allah's Deen, stays away from disagreements in the Ummah, and leaves matters of uncertainty to Allah.

In essence, the 73rd sect that will be saved is the one that embodies true faith, unwavering obedience to the Ahlulbayt, and a deep understanding of their divinely ordained role as guides and protectors of the true message of Islam.

In [46]:
# Ask a question
query = "Tell me bout diff between imaan n islam as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a profound discussion on the essence of Iman and Islam, a conversation he had directly with Imam Ali (AS). He asked Ameerul Momineen (AS), "What is Iman and Islam?"

Imam Ali (AS) illuminated the distinction, stating, "Iman is to understand with knowledge, and accept, and Islam is to accept, bow your head down and obey."

Sulaym then sought further clarification, asking, "Does Iman mean to accept after knowledge (Manfah)?"

Imam Ali (AS) affirmed this, explaining, "He, whom Allah has given knowledge about HIM, His Prophet (PBUH) and His Imam (AS), then he has accepted obedience to Allah – he is a Mumin."

Sulaym then summarized his understanding, "That means from Allah is knowledge (Manfah) and from human being is acceptance."

Imam Ali (AS) confirmed this understanding, elaborating, "Manfah from Allah means that He calls, and gives authority and obliges and gives bounties, and acceptance of Allah means that the human-being accepts that He is free to oblige whomever He pleases and He develops Manfah in whoever’s heart He wishes."

This narration highlights the critical role of both divine guidance and human acceptance in achieving true faith. Knowledge bestowed by Allah (SWT) through His Prophet (PBUH) and the Imams (AS) is the foundation, but it is the individual's conscious acceptance and obedience that completes the essence of Iman. Islam, in this context, is the outward manifestation of that inner conviction, expressed through submission and adherence to Allah's (SWT) commands.

In [ ]:
# Ask a question
query = "Tell me bout the accursed clan as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the narrations in Kitab Sulaym ibn Qays, the animosity towards the Ahlulbayt (AS) is deeply rooted in the hearts of some, stemming from events like the battles of Badr and Uhud. Imam Ali (AS) spoke of those who harbor malice and seek to overpower the rightful successors of the Prophet (PBUH).

Sulaym ibn Qays narrates that Imam Ali (AS) described the state of affairs after the passing of the Prophet (PBUH), saying that the Quraysh had overpowered him and that Umar and his companions had prepared Abu Bakr and paid allegiance to him while he (AS) was occupied with the sacred duties of preparing the Prophet (PBUH) for burial. He (AS) further stated that he reminded the people of his rights and sought their help, but they neither answered nor aided him.

The book also mentions Amr ibn al-Aas, who, after the passing of Ibrahim, the son of the Prophet (PBUH), spoke ill of the Prophet (PBUH) and his lineage. This prompted the revelation of Surah Al-Kauther, which assures the Prophet (PBUH) that his enemy will be without posterity.

Imam Hasan (AS) was forewarned about the treachery he would face from the Ummah. He was advised to fight if he found helpers, but if not, to restrain himself and preserve his blood, as martyrdom awaited him. The Imam (AS) was also informed that his killer would be an illegitimate child, cursed by Allah.

These narrations paint a picture of the trials and tribulations faced by the Ahlulbayt (AS) and their followers, highlighting the importance of remaining steadfast in the face of adversity.

In [49]:
# Ask a question
query = "Tell me about the accursed tree, shajra e maloon as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))      

The "cursed tree" mentioned in the verse, as Imam Ali (AS) explains, is a trial for mankind. Sulaym ibn Qays narrates that Imam Ali (AS) spoke of the Quraysh and their oppression towards them, the Ahlulbayt (AS). He mentioned the verse, "And We did not make the vision which We showed you but a trial for men and the cursed tree in the Qur'an."

This "cursed tree," according to the Imam (AS), refers to those who would usurp the rights of the Ahlulbayt (AS) after the Prophet's (PBUH) passing. They are the ones who would lead people astray and cause great harm to the Ummah. The Holy Prophet (PBUH) informed Imam Ali (AS) that he saw twelve misguiding imams from Quraysh climbed up and down his pulpit, whose faces were like monkeys who are taking his Ummah from Siratul Mustaqeem (the Right Path) back to whom they were.

In [50]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a remarkable event that highlights the unique status of Imam Ali (AS). One morning, Amirul Momineen (AS) went to Baqi' with Abu Bakr, Umar, and a community of Muhajireen and Ansar. He stood on a high place, and as the sun rose, he addressed it, saying, "Salutation, O creature of Allah’s new creation who obeys Him."

According to this tradition, everyone present heard a voice from the sky responding, "And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O batin, O he who knows everything."

The people were astonished and fainted upon hearing this. When they regained consciousness, they went to the Holy Prophet (PBUH) and questioned him about Imam Ali (AS), saying, "You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself."

The Holy Prophet (PBUH) then confirmed the truth of what they had heard. He explained that the sun's words were accurate:

*   "O Awwal": He is the first one who brought faith to me.
*   "O Aakheer": He is the last person of all who will separate from me. He will give me the ritual bath, shroud, and will enter me in my grave.
*   "O Zahir": All my knowledge became Zahir (clear) for him.
*   "O Batin": All my secrets are present with him.
*   "O he, who knows everything": He knows what is lawful and unlawful, and compulsory, tradition, and like it.

This narration underscores the profound connection between the Holy Prophet (PBUH) and Imam Ali (AS), and the immense knowledge and spiritual status bestowed upon him.

In [51]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali also quote the entire incident."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a remarkable event that highlights the unique station of Imam Ali (AS) in the eyes of Allah (SWT) and the Holy Prophet (PBUH).

One morning, after the Fajr prayer, Imam Ali (AS) emerged with Abu Bakr, Umar, and a group of Muhajireen and Ansar. They proceeded to Baqi and Imam Ali (AS) stood upon a high place. As the sun rose, Imam Ali (AS) addressed it, saying, "Salutation, O creature of Allah's new creation who obeys Him."

To the astonishment of all present, a voice from the sky responded, "And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O Batin, O he who knows everything."

Upon hearing this, Abu Bakr, Umar, and many of the Muhajireen and Ansar fainted. When they regained consciousness, they were bewildered and sought an explanation from the Holy Prophet (PBUH). They exclaimed, "You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself."

The Holy Prophet (PBUH) inquired, "What did you hear from it?"

They replied, "We heard it saying 'Salutations be on you, O he who is Awwal.'"

The Holy Prophet (PBUH) confirmed, "It said truth. He is the first one who brought faith to me."

They continued, "We heard it say: 'O Aakheer'."

He replied, "It said truth, he is the last person of all who will separate from me. He will give me the ritual bath, shroud and will enter me in my grave."

They added, "We heard it say: 'O Zahir'."

He replied: "It said truth. All my knowledge became Zahir (clear) for him."

They continued, "We heard it say 'O Batin'."

He replied: "It said truth. All my secrets are present with him."

Finally, they said, "We heard it say 'O he, who knows everything'."

He replied, "It said truth. He knows what is lawful and unlawful, and compulsory, tradition and like it."

This incident, as narrated by Sulaym ibn Qays, serves as a powerful testament to the divinely ordained status of Imam Ali (AS) and the profound knowledge and secrets entrusted to him by the Holy Prophet (PBUH). It underscores his unique position as the first to believe, the last to part, and the inheritor of the Prophet's (PBUH) manifest and hidden knowledge.

In [ ]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali also quote the entire incident and give hadees no and page number from the book."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

Sulaym ibn Qays narrates a remarkable event that highlights the unique station of Imam Ali (AS) in the eyes of Allah (SWT) and His Messenger (PBUH).

One morning, after the Fajr prayer, Imam Ali (AS) emerged, accompanied by Abu Bakr, Umar, and a group of Muhajireen and Ansar. They proceeded to Baqi and Imam Ali (AS) stood upon a high place. As the sun rose, Imam Ali (AS) addressed it, saying, "Salutation, O creature of Allah's new creation who obeys Him."

To the astonishment of all present, a voice from the sky responded, "And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O Batin, O he who knows everything."

Upon hearing this, Abu Bakr, Umar, and many of the Muhajireen and Ansar fainted. When they regained consciousness, they were bewildered and sought an explanation from the Holy Prophet (PBUH). They exclaimed, "You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself."

The Holy Prophet (PBUH) inquired, "What did you hear from it?"

They replied, "We heard it saying 'Salutations be on you, O he who is Awwal.'"

The Holy Prophet (PBUH) confirmed, "It said truth. He is the first one who brought faith to me."

They continued, "We heard it say: 'O Aakheer'."

He replied, "It said truth, he is the last person of all who will separate from me. He will give me the ritual bath, shroud and will enter me in my grave."

They added, "We heard it say: 'O Zahir'."

He replied, "It said truth. All my knowledge became Zahir (clear) for him."

They then said, "We heard it say 'O Batin'."

He replied, "It said truth. All my secrets are present with him."

Finally, they said, "We heard it say 'O he, who knows everything'."

He replied, "It said truth. He knows what is lawful and unlawful, and compulsory, tradition and like it."

This incident, as narrated by Sulaym ibn Qays, serves as a powerful testament to the divinely ordained status of Imam Ali (AS) and the profound knowledge and secrets entrusted to him by the Holy Prophet (PBUH). It underscores his unique position as the first to believe, the last to bid farewell, and the inheritor of the Prophet's (PBUH) manifest and hidden knowledge.

## **LangChain Qdrant**

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
import os 

c:\Users\sadiq\OneDrive\Documents\projects\rag-for-Hayatal-Qulub\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# qdrant setup
qdrant_client = QdrantClient(
    host="xyz-example.eu-central.aws.cloud.qdrant.io",
    api_key=os.getenv("QDRANT_API_KEY"),
)

NameError: name 'QdrantClient' is not defined

In [ ]:
# ---------- begin of pipelines ----------
# load pdf
pdf_path = 'Hayat-Qulub-Alama-Majlisi.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()


In [ ]:
# split text 
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)


In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L12-v2", model_kwargs={"device": "cuda"}
)

In [ ]:
# setup vectordb and store embeddings
vector_db_qdrant = qdrant_client.recreate_collection(
    collection_name="hayat_qulub",
    vectors_config=Qdrant.VectorParams(
         size = 768,
        distance=Qdrant.Distance.COSINE
    ),
)


## **LlamaIndex**

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader 
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
docs = SimpleDirectoryReader('pdf_docs').load_data()
docs


In [ ]:
# splitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter

In [ ]:
embeddings = HuggingFaceEmbedding(model_name="all-MiniLM-L12-v2")
embeddings

In [ ]:
storage_context = StorageContext.from_defaults(index_store=embeddings)
storage_context

In [ ]:
vector_store = VectorStoreIndex.from_documents(docs, storage_context=storage_context)
vector_store